In [1]:
import pandas as pd
import numpy as np
import load_data
from sklearn.model_selection import train_test_split
from ipynb.fs.full.Get_Base_Data_00 import Time

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [2]:
#Load Data
mypath = "../data/team_data/data/"
mydata = load_data.get_file_names(mypath)
df = load_data.load_copy_data(mydata, mypath)

file name: train_month_1
file name: train_month_2
file name: test_month_1
file name: test_month_3
file name: test_month_2
file name: train_month_3_with_target


In [3]:
def merge_df(left_df, right_df, merge_on, how, suffixes=(None,None)):
    df_merge = left_df.merge(right_df, on=[merge_on], how=how, suffixes=suffixes)
    print(f"Shape of dataframe: {df_merge.shape}")
    return df_merge

In [4]:
#Merge data (should be 39+38+39 columns = 118?, rows = 63697) with client_id as key
df_merged = merge_df(df['train_month_1'], df['train_month_2'], 'client_id', 'outer', ('_m1', '_m2'))
df_merged = merge_df(df_merged, df['train_month_3_with_target'], 'client_id', 'outer')


Shape of dataframe: (63697, 77)
Shape of dataframe: (63697, 116)


In [5]:
#Find and Drop duplicated features
def drop_features(col_list, df):
    drop_features = col_list
    
    # Duplicated Columns to drop
    dup_cols = df.T.duplicated().reset_index()
    dup_cols_list = dup_cols.loc[dup_cols[0], 'index'].tolist()
    drop_features.extend(dup_cols_list)

    data = df.drop(drop_features, axis=1)
    
    print(f'Dropped features: {drop_features}')
    print(f'Raw merged data: {df.shape}')
    print(f'Duplicated columns dropped: {data.shape}')
    
    return data

In [6]:
#Drop duplicated features and customer education since it is missing 73% of data
df_merged = drop_features(['customer_education_m1'], df_merged)

Dropped features: ['customer_education_m1', 'has_savings_account_starter_m2', 'customer_since_all_m2', 'customer_since_bank_m2', 'customer_gender_m2', 'customer_birth_date_m2', 'customer_postal_code_m2', 'customer_occupation_code_m2', 'customer_education_m2', 'customer_since_all', 'customer_since_bank', 'customer_gender', 'customer_birth_date', 'customer_postal_code', 'customer_occupation_code', 'customer_education']
Raw merged data: (63697, 116)
Duplicated columns dropped: (63697, 100)


In [7]:
#Construct test, train set
X = df_merged.drop('target',axis=1)
y = df_merged['target']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [8]:
X_train.to_csv(mypath + 'X_train_RAW.csv', encoding='utf-8',index=False)
X_val.to_csv(mypath + 'X_val_RAW.csv', encoding='utf-8',index=False)
y_train.to_csv(mypath + 'y_train_RAW.csv', encoding='utf-8',index=False)
y_val.to_csv(mypath + 'y_val_RAW.csv', encoding='utf-8',index=False)

In [9]:
# Pre-process individual columns

# Change dates to inbetween years
def dates_to_days(df, col_list, base_date = '2018-01-01'):
    #Convert date columns into datetime format
    df['base_dt'] = pd.to_datetime(base_date)
    df[col_list] = df[col_list].apply(pd.to_datetime)

    for col in col_list:
        df[col] = abs(df['base_dt'].dt.year - df[col].dt.year)

    #Drop columns (base_dt)
    df = df.drop('base_dt', axis=1)

    return df

# Categorize Area Code by 1000s
def bin_area_code(df):
    # Bin area codes by 1000s 
    labels = ["{}_area_code".format(i) for i in range(0, 10000, 1000)]
    df['area_cat'] = pd.cut(df['customer_postal_code_m1'], range(0, 10005, 1000), right=False, labels=labels)
    
    return df

# Get % Change from time point 1 to 2, 2 to 3, 1 to 3
# def get_percent_change(df, col_list):
def get_differences(df, col_list):

    def difference(col1,col2):
#     def percentage_change(col1,col2):
#         change = ((col2 - col1) / col1) * 100
        return col2-col1

    for col in col_list:
        df['{}_1'.format(col[2])] = difference(df[col[0]],df[col[1]]) 
        df['{}_2'.format(col[2])] = difference(df[col[1]],df[col[2]]) 
        df['{}_3'.format(col[2])] = difference(df[col[0]],df[col[2]]) 

        df['{}_1'.format(col[2])] = df['{}_1'.format(col[2])].fillna(0)
        df['{}_2'.format(col[2])] = df['{}_2'.format(col[2])].fillna(0)
        df['{}_3'.format(col[2])] = df['{}_3'.format(col[2])].fillna(0)    

    return df

    
# Change NA to category
def categorize_na(df, col_list):
    for col in col_list:
        df[col] = df[col].fillna("Unknown")

    return df

In [10]:
### Pre-process training data ###
print(f'\nStart preproecssing training data.\n')

#1. Change dates to inbetween years
print(f'1. Change dates to number of years.')
list_dates = ['customer_since_all_m1', 'customer_since_bank_m1', 'customer_birth_date_m1']
X_train = dates_to_days(X_train, list_dates, base_date = '2018-01-01')

#2. Categorize Area Code by 1000s
print(f'2. Categorize area code by 1000s.')
X_train = bin_area_code(X_train)

#3. Get % Change from time point 1 to 2, 2 to 3, 1 to 3
print(f'3. Get differences of balances between timepoints.')
list_balances = [['bal_insurance_21_m1', 'bal_insurance_21_m2', 'bal_insurance_21'],
                 ['bal_insurance_23_m1', 'bal_insurance_23_m2', 'bal_insurance_23'],
                 ['bal_personal_loan_m1', 'bal_personal_loan_m2', 'bal_personal_loan'],
                 ['bal_mortgage_loan_m1', 'bal_mortgage_loan_m2', 'bal_mortgage_loan'],
                 ['bal_current_account_m1', 'bal_current_account_m2', 'bal_current_account'],
                 ['bal_pension_saving_m1', 'bal_pension_saving_m2', 'bal_pension_saving'],
                 ['bal_savings_account_m1', 'bal_savings_account_m2', 'bal_savings_account'],
                ]

X_train = get_differences(X_train, list_balances)


# print(f'4. Drop customer_education_m1 and duplicated features.\n')
# X_train = drop_features(['customer_education_m1'], X_train)

print(f'Finished preprocess of training data.\n')



Start preproecssing training data.

1. Change dates to number of years.
2. Categorize area code by 1000s.
3. Get differences of balances between timepoints.
Finished preprocess of training data.



In [11]:
### Pre-process validation data ###
print(f'Start preproecssing validation data.\n')

#1. Change dates to inbetween years
print(f'1. Change dates to number of years.')
X_val = dates_to_days(X_val, list_dates, base_date = '2018-01-01')

#2. Categorize Area Code by 1000s
print(f'2. Categorize area code by 1000s.')
X_val = bin_area_code(X_val)

#3. Get % Change from time point 1 to 2, 2 to 3, 1 to 3
print(f'3. Get differences of balances between timepoints.')
X_val = get_differences(X_val, list_balances)

# print(f'4. Drop customer_education_m1 and duplicated features.\n')
# X_val = drop_features(['customer_education_m1'], X_val)

print(f'Finished preprocess of training data.\n')

Start preproecssing validation data.

1. Change dates to number of years.
2. Categorize area code by 1000s.
3. Get differences of balances between timepoints.
Finished preprocess of training data.



In [12]:
# Export X_train, X_val, y_train, y_val
print(f'Export data to {mypath} as X_train.csv, X_val.csv, y_train.csv, y_val.csv')
X_train.to_csv(mypath + 'X_train_R.csv', encoding='utf-8',index=False)
X_val.to_csv(mypath + 'X_val_R.csv', encoding='utf-8',index=False)
y_train.to_csv(mypath + 'y_train_R.csv', encoding='utf-8',index=False)
y_val.to_csv(mypath + 'y_val_R.csv', encoding='utf-8',index=False)

Export data to ../data/team_data/data/ as X_train.csv, X_val.csv, y_train.csv, y_val.csv
